In [1]:
import numpy as np
import pandas as pd

# The data
The data we are going to use here is the following :

Categorical variables are transformed with get_dummies

ps_car_11_cat is dropped

null values are set to 0

We take 5% of the data obtained with train_test_split, random_state=42

In [2]:
X = np.load('/tmp/X.npy')
X_test = np.load('/tmp/X_test.npy')
y = np.load('/tmp/y.npy')

In [3]:
X.shape, y.shape

((17856, 114), (17856,))

# XGBoost (Binh)

In [4]:
import xgboost as xgb
from sklearn.model_selection import KFold, GridSearchCV

/Users/rafik/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
def gini(pred, y):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def normalized_gini(pred, y):
    return gini(pred, y) / gini(y, y)

# custom normalized gini score for xgb model
def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini_f', normalized_gini(pred, y)

In [6]:
data_test = pd.read_csv('data/test.csv',index_col='id',na_values=-1)

submission = pd.DataFrame(np.zeros(X_test.shape[0]), columns = ['target'])

In [7]:
kfold = 5
nrounds=500
params = {
    'eta': 0.07,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree':0.8,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': True,
    'n_jobs':-1 # use all CPU cores
}

kf = KFold(n_splits=kfold, shuffle=True, random_state=322)

for i, (train_index, cv_index) in enumerate(kf.split(X)):

    print(' xgb kfold: {} of {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index,:], X[cv_index,:]
    y_train, y_eval = y[train_index], y[cv_index]

    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_eval, y_eval)

    watchlist = [(d_train, 'train_f'), (d_valid, 'eval')]
    xgb_model = xgb.train(params,
                          d_train,
                          nrounds,
                          watchlist,
                          early_stopping_rounds=100,
                          feval=gini_xgb,
                          maximize=True,
                          verbose_eval=50)
    # Updating prediction by taking the average
    submission['target'] += xgb_model.predict(xgb.DMatrix(X_test)) / kfold

 xgb kfold: 1 of 5 : 
[0]	train_f-gini_f:0.122802	eval-gini_f:0.142177
Multiple eval metrics have been passed: 'eval-gini_f' will be used for early stopping.

Will train until eval-gini_f hasn't improved in 100 rounds.
[50]	train_f-gini_f:0.645773	eval-gini_f:0.197243
[100]	train_f-gini_f:0.859598	eval-gini_f:0.128816
Stopping. Best iteration:
[33]	train_f-gini_f:0.548779	eval-gini_f:0.231182

 xgb kfold: 2 of 5 : 
[0]	train_f-gini_f:0.143357	eval-gini_f:0.080687
Multiple eval metrics have been passed: 'eval-gini_f' will be used for early stopping.

Will train until eval-gini_f hasn't improved in 100 rounds.
[50]	train_f-gini_f:0.679888	eval-gini_f:0.204816
[100]	train_f-gini_f:0.878969	eval-gini_f:0.177771
[150]	train_f-gini_f:0.948316	eval-gini_f:0.171599
Stopping. Best iteration:
[57]	train_f-gini_f:0.714269	eval-gini_f:0.226559

 xgb kfold: 3 of 5 : 
[0]	train_f-gini_f:0.145399	eval-gini_f:0.028025
Multiple eval metrics have been passed: 'eval-gini_f' will be used for early stoppin

In [8]:
submission.id = submission.id.astype(int)
submission.to_csv('df.csv',index=False)

AttributeError: 'DataFrame' object has no attribute 'id'

In [15]:
submission.shape

(892816, 1)

In [17]:
X_test.shape

(892816, 114)